In [1]:
import cv2 as cv
import numpy as np
import keyboard

import vision

In [2]:
# Vision example on an image

# load the image
img = cv.imread("testing_vision.jpg")
img = cv.resize(img, (640,480), interpolation=cv.INTER_CUBIC) 

# do the actual vision
original_coordinates, new_coordinates = vision.get_fop_coordinates(img)
fop = vision.get_fop(img, original_coordinates, new_coordinates)
start_x, start_y, alpha, width = vision.get_starting_position(fop)
obstacles = vision.get_obstacles(fop,width)
objective_x, objective_y = vision.get_objective(fop)

# draws everything and displays it
output = fop.copy()
cv.circle(output, (objective_x, objective_y), radius=10, color=(255, 0, 0), thickness=-1)
cv.circle(output, (start_x, start_y), radius=10, color=(0, 0, 255), thickness=-1)
for obstacle in obstacles:
    for vertice in obstacle:
        cv.circle(output, (vertice[0],vertice[1]), radius=10, color=(0, 255, 0), thickness=-1)

print("Angle:" + str(alpha))

cv.imshow("image", output)

# press any key to close all windows
cv.waitKey(0) 
cv.destroyAllWindows()

Angle:[2.65711472]


In [3]:
print("Dimensions of the rectangular frame:")
print(new_coordinates[3][0], new_coordinates[3][1])

print("Starting coordinates and angle:") # angle of the robot relative to the x axis, counterclockwise, expressed in radian in range (-pi, pi]
print("(x ,y, alpha) = " + str(start_x) + ", "+ str(start_y) + ", " + str(alpha))

print("Robot width:")
print(width)

print("Vertices of the obstacles:")
print(obstacles)

print("Goal coordinates:")
print(objective_x, objective_y)


Dimensions of the rectangular frame:
580.0 369.0
Starting coordinates and angle:
(x ,y, alpha) = 533, 48, [2.65711472]
Robot width:
62
Vertices of the obstacles:
[[array([502, 134], dtype=int32), array([433, 312], dtype=int32), array([344, 269], dtype=int32), array([372, 140], dtype=int32)], [array([299, 138], dtype=int32), array([193, 281], dtype=int32), array([ 21, 141], dtype=int32), array([18, 92], dtype=int32)]]
Goal coordinates:
42 324


In [7]:
# Global constants here

# width of the video feed
IMAGE_WIDTH = 640
# height of the video feed
IMAGE_HEIGHT = 480
# fps of the video feed
CAMERA_FPS = 30
# keyboard input to start
KEYBOARD_INPUT = "enter"
# number of frame thrown away to allow the camera to focus in the meantime
CAMERA_REFRESH_TIME = 30

In [8]:
# 0 is laptop webcam, 1 is USB camera (!COMPUTER DEPENDENT!)
cap = cv.VideoCapture(0)

init = False # boolean set to True on keyboard input to start initialisation
is_init = False # boolean set to True once initialisation has succeeded
black_frame_remaining = 0 # numbers of frames to throw away

# variable initialized at False irrelevant of wether it actually is
camera_hidden = False

# if unable to connect to the camera
if not (cap.isOpened()):
    print("Could not open video device")
else:
    while(True):
        # get the input frame by frame (shape (480,640,3))
        ret, frame = cap.read() 
        cap.set(cv.CAP_PROP_FPS, CAMERA_FPS) 
        frame = cv.resize(frame, (IMAGE_WIDTH,IMAGE_HEIGHT), interpolation=cv.INTER_CUBIC) 

        # wait until keyboard input to initialize vision
        if keyboard.is_pressed(KEYBOARD_INPUT):
            init = True
        
        # initialize vision: start, obstacles and goal coordinates
        if init == True:

            try:
                original_coordinates, new_coordinates = vision.get_fop_coordinates(frame)
                fop = vision.get_fop(frame, original_coordinates, new_coordinates)
                start_x, start_y, alpha, width = vision.get_starting_position(fop)
                obstacles = vision.get_obstacles(fop, width)
                objective_x, objective_y = vision.get_objective(fop)
                is_init = True
            except:
                print("Initialisation failed")

            init = False
            
        if is_init:  
        ################################## CALL FUNCTIONS HERE ################################## 
            
            fop = vision.get_fop(frame, original_coordinates, new_coordinates)
            center_x, center_y = vision.get_robot_center(fop)
            
            # camera was covered, now uncovered
            if center_x != 0 and center_y != 0 and camera_hidden:
                camera_hidden = False
                black_frame_remaining = CAMERA_REFRESH_TIME # to give it time to gain focus

            # camera was uncovered, now covered
            elif center_x == 0 and center_y == 0:
                camera_hidden = True

            # camera uncovered and usable
            elif black_frame_remaining <= 0:
                fop = cv.circle(fop, (center_x,center_y), radius=10, color=(0, 255, 0), thickness=-1)
                
            else:
                # set the first frames after uncovering to black to allow time to focus
                fop = cv.cvtColor(fop, cv.COLOR_BGR2GRAY)
                _, fop = cv.threshold(fop,255,255,cv.THRESH_BINARY)
                black_frame_remaining -= 1

        #########################################################################################
        else:
            # to output the raw feed
            fop = frame

        cv.imshow('preview',fop)
        
        # Close all windows and get out of the loop if ESC is pressed
        if cv.waitKey(1) == 27:
            cv.destroyAllWindows()
            break